  ## Applying Functions to Data

  Data visualization is often the first step on the statistical journey to explore a research question. However, this is usually not where the journey stops, instead additional analyses are often performed to learn more about the average trends seen in the data. These can often be split into two broad categories,

  1. Descriptive Statistics
  2. Inferential Statistics

  **Descriptive Statistics** help to describe the data and are particularly useful to give a single numeric summary for a single variable. We will explore this idea more fully in this section.
  **Inferential Statistics** help us to make broader statements from the data we have to the larger group of interest, commonly referred to as the population. More details on these steps later in the course.

  ## Setup

  We are going to use some real data about higher education institutions from the college scorecard (https://collegescorecard.ed.gov/) to explore the types of conclusions we can make from the data. The college scorecard releases data on higher education institutions to help make the institutions more transparent and provide a place for parents, students, educators, etc can get information about specific instituations from a third party (i.e. US Department of Education).

  ### Loading R packages

In [0]:
.libPaths('../RPackages')

update.packages(ask = FALSE)

library(tidyverse)
library(ggformula)
library(mosaic)

theme_set(theme_bw())


  ### Read in Data

  The below code will read in the data for us to use in the future. The R function to read in the data is `read_csv()`. Function arguments are passed within the parentheses and for the `read_csv()` function the first argument is the path to the data. The data for this example are posted on GitHub in a comma separated file. This means the data is stored in a text format and each variable (i.e. column in the data) is separated by a comma. This is a common format data is stored.

  The data is stored to an object named `college_score`. In R (and other statistical programming languages), it is common to use objects to store results to use later. In this instance, we would like to read in the data and store it to use it later. For example, we will likely want to explore the data visually to see if we can extract some trends from the data. The assignment to an object in R is done with the `<-` assignment operator. Finally, there is one additional argument, `guess_max` which helps to ensure that the data are read in appropriately. More on this later.

In [0]:
college_score <- read_csv("https://raw.githubusercontent.com/lebebr01/statthink/master/data-raw/College-scorecard-4143.csv", guess_max = 10000)
head(college_score)


  ## Functions to columns of data

  Data are often stored in a tabular format where the rows of the data are the units and the columns in a data frame are the varaibles. This is shown in the college scorecard data above where the rows of the data are specific institutions of higher education and the columns represent various attributes about those higher education institutions. This is a common structure to store data where each row represents a unique unit or measurement occasion for longitudinal data.

  In the data visualization units, we accessed columns of data to view the distribution of the particular variable. For example, we explore histograms of admission rate. Instead of visualizing the data, now we will apply functions to these columns to calculate statistics of interest. In particular, the focus will be on the calculating statistics for variables that are numeric rather than representing categories. We will discuss this in more detail as we move along.

  Let's keep talking about the admission rate as we have explored that visually already and start with an example.

In [0]:
df_stats(~ adm_rate, data = college_score, median)



  The `df_stats()` function takes a formula syntax that is similar to the syntax used for viewing a univariate distribution you saw earlier. In particular, the variable that we wish to compute a statistic on is specified after the `~`. The next argument is the data argument. Finally, subsequent arguments after the data argument are functions that we want to compute for the variable specified. Here, I compute the median which happens to be, 0.708. The median is also referred to as the 50% percentile and is the location where half of the data (in this case higher education institutions) are above and below an admission rate of 70.8%.

  Let's think where this shows up on the admission rate distribution we plotted earlier.

In [0]:
gf_histogram(~ adm_rate, data = college_score, bins = 30) %>%
  gf_vline(color = 'blue', xintercept = ~df_stats(~ adm_rate, data = college_score, median)[[1]], size = 1)


  You'll notice that the line is just to the left of the main peak of the data. Does it appear that half of the data are below and half are above the blue line in the figure?

  The median is a special percentile, however other percentiles may be of interest. For example, maybe we'd want to know what the 20th percentile is or the 80th percentile to apply to a school that isn't too selective or is not selective at all. We can compute these with the `df_stats()` function again.

In [0]:
q <- college_score %>%
  df_stats(~ adm_rate, quantile(c(0.2, 0.5, 0.8)), nice_names = TRUE)
q


  Let's look where these fall on our distribution.

In [0]:
gf_histogram(~ adm_rate, data = college_score, bins = 30) %>%
  gf_vline(color = 'blue', xintercept = ~ value, data = gather(q), size = 1)


  Does it appear that 20% of the data are below the first line and 20% are above the last line? Difficult to view on the histogram. An empirical distribution figure, sometimes called an ogive, can be helpful to show these.

In [0]:
gf_ecdf(~ adm_rate, data = college_score) %>%
  gf_vline(color = 'blue', xintercept = ~ value, data = gather(q), linetype = 2) %>%
  gf_hline(color = 'darkblue', yintercept = ~c(0.2, 0.5, 0.8), data = NA, linetype = 3) %>%
  gf_labs(y = 'Cumulative proportion')


  Here you can see that the horizontal lines cross over the vertical lines at the specified values (i.e. 20% for the first vertical line, 50% for the second vertical line, 80% for the final vertical line).

  ## Considering Groups
  We've spent a lot of time trying to reason about other variables that may be important in explaining variation in our variable of interest. So far we have only explored the variable without considering other variables, in practice that is not that useful.

  Instead, it is common to compute conditional statistics based on other characteristics in the data. An example may help to show the idea more clearly.

In [0]:
college_score %>%
  df_stats(adm_rate ~ region, median)


  Presented above are the conditional medians for the higher education institutions in different areas of the country. More specifically, the data are essentially split into subgroups and the median is computed for each of those subgroups instead of pooling all institutions into a single data frame. The formula syntax is now `outcome ~ grouping` where the variable of interest (i.e. commonly a numeric variable) and the variable to the right of the `~` is the grouping variable. This syntax is similar to the violin plots that were created earlier.

  Can you see differences in the admission rates across the regions?

  One thing that is useful to add in when computing conditional statisics, is how many data points are in each group. This is particularly useful when the groups are different sizes, which is common. To do this, we can add another function to the `df_stats()` function.

In [0]:
college_score %>%
  df_stats(adm_rate ~ region, median, length)


  This adds another columns which represents the number of observations that went into the median calculation for each group. The syntax above also shows that you can add additional functions separated by a comma in the `df_stats()` function and are not limited to a single function. We will take advantage of this feature later on.

  ### Adding additional groups
  What if we thought more than one variable was important in explaining variation in the outcome variable? These can also be added to the `df_stats()` function for additional conditional statistics. The key is to add another variable to the right-hand side of the formula argument. More than one variable are separated with a `+` symbol.

In [0]:
college_score %>%
  df_stats(adm_rate ~ region + preddeg, median, length)


  ## Other statistics of center
  So far we have been discussing the median. The median attempts to provide a single number summary for the center of the distribution. It is a robust statistic, but likely isn't the most popular statistic to provide a location for the center of a distribution. The mean is often more commonly used as a measure of the center of a distribution. Part of this is due to the usage of the mean in common statistical methods and the mean also uses the values of all the data in the calculation. The median only considers the values of the middle score or scores, therefore this statistic is less sensitive to extreme values than the mean. I like to look at both statistics and this can provide some insight into the distribution of interest. We can add the mean using the `df_stats()` function by adding the function `mean`.

In [0]:
stats_compute <- college_score %>%
  df_stats(adm_rate ~ region, median, mean, length)
stats_compute


  Do you notice any trends in the direction the mean and median typically follow? More specifically, is the mean typically larger than the median or vice versa?

 Let's visualize them.

In [0]:
gf_histogram(~ adm_rate, data = college_score, bins = 30) %>%
  gf_facet_wrap(~ region) %>%
  gf_vline(color = 'blue', xintercept = ~ median_adm_rate, data = stats_compute, size = 1) %>%
  gf_vline(color = 'lightblue', xintercept = ~ mean_adm_rate, data = stats_compute, size = 1)


  What is different about the distributions that have larger differences in the mean and median?